In [87]:
# Data processing
import pandas as pd
import numpy as np
import ssl

# Text preprocessiong
import nltk
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS

# Topic model
from bertopic import BERTopic

# Dimension reduction
from umap import UMAP

In [28]:
# disable ssl check (to be able to download nltk packages)

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [29]:
# download nltk packages

nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cyrille/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /Users/cyrille/nltk_data...
[nltk_data] Downloading package wordnet to /Users/cyrille/nltk_data...


In [5]:
with open('data/transcripts/transcript_5001.csv', encoding='utf-8') as file:
    transcript_df = pd.read_csv(file).drop(columns='Unnamed: 0')
    
transcript_df

,ID,IdSubject,PersonNumber,Text,MeetingCouncilAbbreviation,MeetingDate,IdSession,SpeakerFunction,CouncilId,Start,End,Function,LanguageOfText
0,191147,36015,214,"Präsident (Stamm Luzi, Alterspräsident): Frau ...",N,20151130,5001,Mit-M,1.0,2015-11-30T14:31:12,2015-11-30T14:54:17,Mit-M,NaN
1,191153,36015,4186,"""Un politico guarda alle prossime elezioni. Un...",N,20151130,5001,Mit-F,1.0,2015-11-30T14:54:17,2015-11-30T15:03:07,Mit-F,IT
2,191155,36015,214,"Präsident (Stamm Luzi, Alterspräsident): Ich b...",N,20151130,5001,Mit-M,1.0,2015-11-30T15:03:07,2015-11-30T15:03:46,Mit-M,DE
3,191152,36016,214,"Präsident (Stamm Luzi, Alterspräsident): Dem A...",N,20151130,5001,Mit-M,1.0,2015-11-30T15:03:46,2015-11-30T15:05:12,Mit-M,DE
4,191157,36016,519,Zur Konstituierung des Rates: Sie haben den Be...,N,20151130,5001,Mit-M,1.0,2015-11-30T15:05:12,2015-11-30T15:07:12,B,DE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1438,193945,36352,525,Der Bundesrat hat in seiner Stellungnahme ein ...,S,20151217,5001,Mit-M,2.0,2015-12-17T10:22:05,2015-12-17T10:23:45,Mit-M,DE
1439,193930,36357,825,Vorab möchte ich mich insofern für den Vorstos...,S,20151217,5001,Mit-M,2.0,2015-12-17T10:24:58,2015-12-17T10:32:25,Mit-M,DE
1440,193924,36357,4055,Auch ich möchte eine ergebnisoffene Diskussion...,S,20151217,5001,Mit-M,2.0,2015-12-17T10:32:25,2015-12-17T10:37:45,Mit-M,DE
1441,193934,36357,3921,Auch ich bin ein bisschen überrascht von diese...,S,20151217,5001,Mit-M,2.0,2015-12-17T10:37:45,2015-12-17T10:40:54,Mit-M,DE


In [68]:
filtered_transcript = transcript_df.loc[transcript_df['LanguageOfText'] == 'FR'][['ID', 'Text']]
filtered_transcript

,ID,Text
5,191151,Vous avez reçu le rapport du Conseil fédéral s...
9,191158,Pour adopter les décisions et formuler les pro...
18,191267,Vous vous souvenez que le groupe UDC recommand...
21,191199,"Dans ce dossier, nous sommes maintenant à bout..."
28,191226,On peut rester relativement calme sur ce sujet...
...,...,...
1426,193850,"Madame Bruderer Wyss, je peux en effet confirm..."
1428,193899,Le but de mon interpellation n'est pas d'ajout...
1430,193929,"En préambule, j'aimerais saluer l'interpellati..."
1432,193855,L'apprentissage des langues est une question q...


In [84]:
for idx, row in filtered_transcript.iloc[100:101].iterrows():
    print(row['Text'])
    print('----')

Avez-vous lu le message du Conseil fédéral sur ce sujet? Si c'est le cas, eh bien vous avez perdu votre temps! Pourquoi? Parce qu'il est complètement dépassé. A l'origine, ce message était prévu comme contre-projet indirect à l'initiative populaire "pour une caisse publique d'assurance maladie". Dans l'intervalle, beaucoup de choses ont été votées au sein de ce Parlement.
Premier élément, la question de l'affinement de la compensation des risques. Nous avons voté sur ce sujet au Parlement lors de la dernière législature. Tout le volet concernant la compensation des risques de ce message est donc dépassé.
Deuxième élément, la loi fédérale sur la surveillance de l'assurance-maladie sociale. Ici aussi, nous avons adopté au sein de ce Parlement cette loi qui entrera bientôt en vigueur. Si vous lisez la page 7140 du message, vous verrez que tous les problèmes pouvant survenir, notamment au niveau des coûts administratifs ou des subventionnements croisés, sont décrits. Vous pouvez donc aussi

In [70]:
filtered_transcript.info()

<class 'pandas.core.frame.DataFrame'>
Index: 324 entries, 5 to 1436
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      324 non-null    int64 
 1   Text    324 non-null    object
dtypes: int64(1), object(1)
memory usage: 7.6+ KB


In [124]:
nlp = spacy.load("fr_core_news_sm")
lemmatizer = nlp.get_pipe("lemmatizer")
nlp.Defaults.stop_words |= {'-t', 'avez', 'conseil', 'commission', 'fédéral', 'suisse', 'être', 'loi', 'projet'}
print(nlp.Defaults.stop_words)

{'etais', 'attendu', 'quelques', 'allaient', 'excepté', 'spécifiques', 'neanmoins', 'o', 'tout', 'meme', 'septième', 'â', 'restent', 'tenant', 'as', 'desquelles', 'ha', 'autrui', 'desquels', 'ont', 'sept', 'tels', 'toi-même', 'étaient', 'quarante', 'suis', 'permet', 'qui', 'déja', 'troisièmement', 'un', 'huitième', 'quiconque', 'puis', 'suffisante', 'et', 'soi', 'depuis', 'specifique', 'te', 'pourquoi', 'nos', 'bat', 'ci', 'vé', 'ceux-là', 'on', 'suivants', 'doivent', 'sinon', 'ainsi', 'devant', 'revoila', 'certes', 'avaient', 'hé', 'tienne', 'eu', 'surtout', 'certaine', 'chez', 'lès', 'suivant', 'voila', 'deja', 'soi-meme', 'durant', 'vu', 'effet', 'ceci', 'au', 'tien', 'celle', 'plutot', 'mien', 'environ', 'quelles', 'précisement', 'suit', 'dire', 'dont', 'merci', 'tu', 'deuxième', 'gens', 'miens', 'diverse', 'selon', 'seule', 'da', 'nous', 'ah', 'apres', 'après', 'anterieur', 'plus', 'm’', 'parlent', 'i', 'etc', 'pres', 'vous', 'hem', 'notre', 'quoique', 'facon', 'afin', 'aie', 'pou

In [131]:
text = filtered_transcript.iloc[100]['Text']
print(text)
print('---')

doc = nlp(text)
lemma_list = [token.lemma_ for token in doc if not any([token.is_stop, token.is_punct, token.is_space])]
' '.join(lemma_list)

Avez-vous lu le message du Conseil fédéral sur ce sujet? Si c'est le cas, eh bien vous avez perdu votre temps! Pourquoi? Parce qu'il est complètement dépassé. A l'origine, ce message était prévu comme contre-projet indirect à l'initiative populaire "pour une caisse publique d'assurance maladie". Dans l'intervalle, beaucoup de choses ont été votées au sein de ce Parlement.
Premier élément, la question de l'affinement de la compensation des risques. Nous avons voté sur ce sujet au Parlement lors de la dernière législature. Tout le volet concernant la compensation des risques de ce message est donc dépassé.
Deuxième élément, la loi fédérale sur la surveillance de l'assurance-maladie sociale. Ici aussi, nous avons adopté au sein de ce Parlement cette loi qui entrera bientôt en vigueur. Si vous lisez la page 7140 du message, vous verrez que tous les problèmes pouvant survenir, notamment au niveau des coûts administratifs ou des subventionnements croisés, sont décrits. Vous pouvez donc aussi

'vous lire message sujet cas bien perdre temps complètement dépasser origine message prévoir contre-projet indirect initiative populaire caisse public assurance maladie intervalle beaucoup chose être voter sein parlement élément question affinement compensation risque voter sujet parlement dernier législature volet compensation risque message dépasser élément fédéral surveillance assurance maladie social ici adopter sein parlement entrer bientôt vigueur lire page 7140 message voir problème pouvoir survenir niveau coût administratif subventionnement croiser décrire pouvoir oublier passage fédéral surveillance assurance maladie social résoudre problème point message question séparation juridique assurance maladie base assurance complémentaire point examiner vision estimer falloir assurance maladie base assurance complémentaire véritable muraill Chine vouloir vouloir facture séparer courrier séparer système informatique séparer bien coût 400 million franc 200 million part ressortir auditi

In [31]:
# Remove stopwords
stopwords = nltk.corpus.stopwords.words('french')
print(f'There are {len(stopwords)} default stopwords. They are {stopwords}')

There are 157 default stopwords. They are ['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont'

In [125]:
# remove stopwords, punctuation and then lemmatize
filtered_transcript['text_lemmatized'] = filtered_transcript['Text'].apply(
    lambda x: ' '.join([token.lemma_ for token in nlp(x) if not any([token.is_stop, token.is_punct, token.is_space])])
)
# Take a look at the data
filtered_transcript

,ID,Text,text_lemmatized
5,191151,Vous avez reçu le rapport du Conseil fédéral s...,recevoir rapport élection national rapport bur...
9,191158,Pour adopter les décisions et formuler les pro...,adopter décision formuler proposition figurer ...
18,191267,Vous vous souvenez que le groupe UDC recommand...,souvenir groupe UDC recommander entrer matière...
21,191199,"Dans ce dossier, nous sommes maintenant à bout...",dossier être bout nouvellement élu possibilité...
28,191226,On peut rester relativement calme sur ce sujet...,rester calme sujet contrairement passer dernie...
...,...,...,...
1426,193850,"Madame Bruderer Wyss, je peux en effet confirm...",monsieur Bruderer Wyss confirmer figurer répon...
1428,193899,Le but de mon interpellation n'est pas d'ajout...,but interpellation ajouter ligne liste long in...
1430,193929,"En préambule, j'aimerais saluer l'interpellati...",préambule aimer saluer interpellation collègue...
1432,193855,L'apprentissage des langues est une question q...,apprentissage langue question revenir régulièr...


In [132]:
# Initiate UMAP
umap_model = UMAP(
    n_neighbors=15, 
    n_components=10, 
    min_dist=0.0, 
    metric='cosine', 
    random_state=100,
)

# Initiate BERTopic
topic_model = BERTopic(language="french", umap_model=umap_model, calculate_probabilities=True)

# Run BERTopic model
docs = filtered_transcript['text_lemmatized'].to_list()
topics, probabilities = topic_model.fit_transform(docs)

2023-05-11 17:07:59,357 - BERTopic - Transformed documents to Embeddings
2023-05-11 17:08:00,162 - BERTopic - Reduced dimensionality
2023-05-11 17:08:00,174 - BERTopic - Clustered reduced embeddings


In [133]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,117,-1_être_initiative_faire_question
1,0,58,0_franc_budget_million_proposition
2,1,33,1_rente_av_avs_pilier
3,2,31,2_armée_service_di_frontière
4,3,30,3_cas_être_public_droit
5,4,29,4_canton_être_langue_national
6,5,26,5_patient_médecin_maladie_médicament


In [128]:
# Get the list of topics
#topic_model.get_topic(4)
topic_model.get_document_info(docs).iloc[100]

Document                   vous lire message sujet cas bien perdre temps ...
Topic                                                                      3
Name                                              3_cas_être_public_question
Top_n_words                cas - être - public - question - assurance - p...
Probability                                                         0.224398
Representative_document                                                False
Name: 100, dtype: object